In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 10
Maxiterations_turbine = 10

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]


time_ac = 3600
PKM_zaryad = True
PKM_razryad = False
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 20
pkm_pgu_tol = 10**-1
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
 
arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
]


Calculate_CCGT_PKM_iter(arguments_all_it,Iter_pkm,pkm_pgu_tol)

-ТЕПЛА ОТ ПКМ НЕ ХВАТАЕТ НА ТЕПЛОФИКАЦИЮ 149294.73292942205 / 164369.78715262012
Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций расхода КУ+ПТУ
Error_water_G: 1.1206346425898842e-14, Error_nd_G: 1.7540585310418662, Error_vd_G: 0.0
Время 1 итерации расчета КУ+ТУ:---  24.7 сек. ---
Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций давления верхнего отбора
Невозможно получить необходимое давление, диафрагма полностью открыта
Недостаточное количество итераций
Достигнуто максимальное количество итераций давления верхнего отбора
Невозможно получить необходимое давление, диафрагма полностью открыта
Недостаточное количество итераций
Достигнуто максимальное количество итераций расхода КУ+ПТУ
Error_water_G: 7.285742413229232e-13, Er

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.010558,0.1,958.533980,503.497387,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,81.662904,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,581.494637,0.1,1038.442584,585.160292,0.770331,0.107499,0.038709,0.076655,0.006805
PEVD-IVD,488.610887,0.1,930.390135,585.160292,0.770331,0.107499,0.038709,0.076655,0.006805
IVD-EVD,278.764327,0.1,694.828833,585.160292,0.770331,0.107499,0.038709,0.076655,0.006805
EVD-PPND,207.359720,0.1,617.251203,585.160292,0.770331,0.107499,0.038709,0.076655,0.006805
PPND-IND,205.113347,0.1,614.829225,585.160292,0.770331,0.107499,0.038709,0.076655,0.006805
IND-GPK,155.889858,0.1,562.012935,585.160292,0.770331,0.107499,0.038709,0.076655,0.006805
GPK-out,102.311423,0.1,505.014000,585.160292,0.770331,0.107499,0.038709,0.076655,0.006805


In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,15.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.1,0.1,958.869199,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
REF-SMESH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-PEVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PEVD-IVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IVD-EVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EVD-PPND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PPND-IND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IND-GPK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GPK-out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
water_streams

,T,P,H,G,S,X
AIR,15.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,542.875789,4.170043,3542.403894,83.447102,NaN,NaN
IVD-PEVD,266.589371,5.215342,2792.359240,83.447102,NaN,NaN
EVD-IVD,264.450975,5.215342,1157.160634,83.447102,NaN,NaN
PEN-EVD,146.147357,5.215342,618.599898,83.447102,NaN,NaN
BND-PEN,145.353840,0.419750,612.160640,83.447102,NaN,NaN
PPND-DROSND,188.459073,0.411163,2835.985759,14.653105,NaN,NaN
IND-PPND,145.353840,0.419750,2740.240784,14.653105,NaN,NaN
GPK-IND,145.353840,0.419750,618.126423,98.100206,NaN,NaN
GPK-REC,145.353840,0.419750,618.126423,98.100206,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3683.821929,73.571863,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3023.629730,73.571863,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.0,2.0,2545.703624,73.571863,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Sgaccum,100.0,2.0,746.793127,73.571863,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
SGaccum-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SGaccum-GTU,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR-COMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COMP-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COMB-TURB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TURB-COOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
electric

,Ni,N,KPDm,KPD
PEN,0.537337,0.611812,0.878273,0.807400
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,150.750600,NaN,33.348086
DK,NaN,6.269747,NaN,NaN
Turbine,23.079901,NaN,NaN,NaN
Tots1,8.541177,NaN,NaN,0.836795
Tots2,4.129946,NaN,NaN,0.836906
Tots3,2.391101,NaN,NaN,0.814864
Tots4,8.017678,NaN,NaN,0.766770


In [6]:
heaters

,Qw,Qg,KPD
PEVD,62595.722494,63228.002519,0.99
IVD,136462.709327,137841.120533,0.99
EVD,44941.395041,45395.348527,0.99
PPND,1403.072798,1417.245250,0.99
IND,30596.935950,30905.995909,0.99
GPK,33019.978166,33353.513299,0.99
SP2,0.000000,NaN,0.00
SP1,0.000000,NaN,0.00
OD,0.000000,NaN,0.00
Strans,216395.422859,NaN,NaN


In [7]:
########################ОГРАНИЧЕНИЯ НА РАБОТУ ПГУ########################
# ЕСЛИ ТЕПЛОФИКАЦИЯ (из документов по ПГУ-220Т)
# Gк_мин=4,44 кг/с вроде
# Nтурбины мин = 17,6 МВт (примерно 25%)
# Максимальноа давление в отборах СП 0,245, 0,198 МПа (Теплофикационная паровая турбина Т-63/76-8.8 для серии ПГУ-230)
# ДЛЯ ВСЕХ РЕЖИМОВ (Из Трухния по ПГУ-450Т)
# Gвд_мин=25% от номинала
# Gнд_мин = 50% от номинала


########РАСЧЕТ расзода пара на ПКМ из условия заполнения хранилища################

GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)

Max_iterations_minimum = 10

if Сalculate_minimum == True:
    n_GTU = GTU_input.at["n", 1]
    start_time = time.time()
    for i in range(Max_iterations_minimum):
        print(f"Началась {i+1} итерация расчета ПГУ")
        if i < round(Max_iterations_minimum / 2, 1):
            New_iterations_KU_TU, New_iterations_cotel, New_iterations_turbine = (
                2,
                2,
                15,
            )
        else:
            New_iterations_KU_TU, New_iterations_cotel, New_iterations_turbine = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
            )

        Delt_Gcnd = 100
        Delt_Nturb = 100
        if Teplo == 1:
            Delt_Gcnd = (water_streams.at["INKOND", "G"] - 4.44) / 4.44
            Delt_Nturb = (electric.at["Turbine", "Ni"] - 17.6) / 17.6
        Delt_Gvd = (
            water_streams.at["PEVD-DROSVD", "G"] / water_streams0.at["PEVD-DROSVD", "G"]
            - 0.25
        ) / 0.25
        Delt_Gnd = (
            water_streams.at["PPND-DROSND", "G"] / water_streams0.at["PPND-DROSND", "G"]
            - 0.5
        ) / 0.5
        Delta_min = min(Delt_Gcnd, Delt_Nturb, Delt_Gvd, Delt_Gnd)
        if n_GTU == 1 and Delta_min < 0:
            print("Мощность ГТУ 100% и расход пара все еще слишком мал")
        n_GTU = n_GTU - Delta_min / 10
        GTU_input.at["n", 1] = n_GTU

        calculate_CCGT(
            New_iterations_KU_TU, New_iterations_cotel, New_iterations_turbine
        )
        print(f"Отклонение от ограничения минимальное равно {Delta_min}")
        if abs(Delta_min) < Calctolerance:
            calculate_CCGT(
                New_iterations_KU_TU, New_iterations_cotel, New_iterations_turbine
            )
            print(f"Отклонение от ограничения минимальное равно {Delta_min}")
            print(f"Относительная мощность ГТУ равна {n_GTU}")
            print(
                f"fin минимальная мощность ПГУ:--- {round((time.time() - start_time), 1)} сек. ---"
            )
            break
        if i == Max_iterations_minimum - 1:
            print(
                "Достигнуто максимальное количество итераций минимального расхода в ПГУ",
                i + 1,
            )

Началась 1 итерация расчета ПГУ


NameError: name 'Teplo' is not defined